In [ ]:
#先import需要的库函数
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from touch.utils.data import DataLoader,Dataset
import time

In [ ]:
#定义读取图片文件函数
def readfile(path,flag):
    """
    path:图片所在文件夹的路径
    flag:为True时是training set或validation set,为False时是testing set
    return值为数值化后的图片数据
    """
    image_dir=os.listdir(path) #将path路径指定的文件夹内的所有文件路径以列表形式返回
    #先创建两个规定大小的全零array
    x=np.zeros((len(image_dir),128,128,3) #3为通道数
    y=np.zeros((len(image_dir))) #x为所有图片叠加成的一个矩阵，y则为所有图片的标签形成的矩阵
    for i,file in enumerate(image_dir): #遍历每一张图片，并将x，y内元素赋值，enumerate用于将一个可遍历组合为一个索引序列
        img=cv2.imread(os.path.join(path,file)) 
        #os.path.join()用于将两个路径拼接，如此出path是图片文件夹路径，file就是图片名，组合起来就是每张图片路径
        x[i,:,:]=cv2.resize(img,(128,128)) #图片赋值到x中前，由于图片size不同，都变成128*128的
        if flag: #training或alidation
            y[i]=int(file.split('_')[0]) #截取图片名中种类的那部分
    if flag: #training和validation
        return x,y
    else: #testing
        return x

In [ ]:
#用上面readfile函数把training set,validation set,testing set读进来
workspace_dir='/home/zjy/ml hw3/food-11' #用于后面路径拼接
print("Reading data")
train_x,train_y=readfile(os.path.join(workpace_dir,"training"),True)
print("Size of training data = {}".format(len(train_x)))
val_x,val_y=readfile(os.path.join(workpace_dir,"validation"),True)
print("Size of validation data = {}".format(len(val_x)))
test_x=readfile(os.path.join(workpace_dir,"testing"),False)
print("Size of testing data = {}".format(len(test_x)))

In [ ]:
#对data做augmentation数据增强,即通过翻转旋转图片等来获得更多的training data，同时将格式转换成tenser形式
train_transform=transforms.Compose([ #对training data的transform，compose是用于将多个变换打包在一起
    transforms.ToPILImage()，#图像转为pil
    transforms.RandomHorizontalFlip(), #随机将图像水平翻转
    transforms.RandomRotation(15), #随机将图像旋转15度
    transforms.ToTensor(), #将图像变成tensor并normalize到[0,1]
])
#testing data不用data augmentation
test_transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

In [ ]:
"""
使用Dataset和DataLoader来打包包装data，分成多个batch后再输出，使后续testing更方便
Dataset是用来继承的父类，可以实现对数据的封装，继承了Dataset后要重写len和getitem这两个方法
len方法是给出dataset的大小，getitem支持索引从0到len(self)的数据
DataLoader是用过getitem函数获得单个数据并可以组合成batch
"""
class ImagDataset(Dataset): 
"""
类名开头大写,__init__初始化函数，在定义对象时，对象的实参对应初始化函数的形参
self可看成对象，所有类的方法形参第一个都要是self，在写类的属性时也要用self.
"""
    def __init__(self,x,y=None,transform=None): 
        self.x=x
        self.y=y
        if y is not None: #label要是longtensor形式
            self.y=torch.LongTensor(y)
        self.transform=transform
    def __len__(self)：
        return len(self.x)
    def __getitem__(self,index):
        X=self.x[index]
        if self.transform is not None:
            X=self.transform(X)
        if self.y is not None:
            Y=self.y[index]
            return X,Y
        else:
            return X

In [ ]:
#接下来调用上面的定义来进行分批，并用Dataloader完成最后的包装
batch_size=64 #设置bach_size
train_set=ImgDataset(train_x,train_y,train_transform)
val_set=ImgDataset(val_x,val_y,test_transform)
train_loader=DataLoader(train_set,batch_size=batch_size，shuffle=True) #shuffle是用于决定每次迭代训练时是否对数据洗牌
val_loader=DataLoader(val_set,batch_size=batch_size,shuffle=False)

In [ ]:
#进行网络的搭建
"""
构建网络需要继承nn.Module父类，并调用nn.Module的构造函数
有两个module，一个是cnn卷积层，一个是fc即fully connected层，fc层是在cnn出来后flatten再进入
cnn层要利用nn.Conv2d,nn.BatchNorm2d,nn.ReLU,nn.MaxPool2d这四个函数构建一个五层的cnn
BatchNorm2d层一般是跟在Conv2d层后做归一化
"""
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__() #super().__init__()作用就是让我们可以执行父类的构造函数同时调用父类的属性
        """
        以下列出要用函数的形参
        torch.nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
        torch.nn.MaxPool2d(kernel_size,stride,padding)
        以上kernel_size就是window的size，padding时边缘是否填0
        """
        #卷积层
        self.nn=nn.Sequential(
            #卷积层1
            nn.Conv2d(3, 64, 3, 1, 1),  #output：[64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #output:[64, 64, 64]
            #卷积层2
            nn.Conv2d(64, 128, 3, 1, 1),  #output：[128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #output：[128, 32, 32]
            #卷积层3
            nn.Conv2d(128, 256, 3, 1, 1),  #output：[256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #output：[256, 16, 16]
            #卷积层4
            nn.Conv2d(256, 512, 3, 1, 1),  #output：[512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #output：[512, 8, 8]
            #卷积层5
            nn.Conv2d(512, 512, 3, 1, 1),  # utput：[512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #output：[512, 4, 4]
        )
        #fully connected层
        self.fc = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        #pytorch和keras不同，keras中flatten函数是和卷积一起写的，但pytorch中是要单独在forward中用view写
            def forward(self,x): #forward将上两个网络调用
                out=self.cnn(x)
                out=out.view(out.size()[0],-1) #flatten
                return self.fc(out)

In [ ]:
#train模型
model=Classifier().cuda() #使用gpu计算
loss=nn.CrossEntropyLoss() #Loss用crossentropy
optimizer=torch.optim.Adam(model.parameters(),lr=0.001) #optimizer是自动做gradient descent的优化器，并在此处指定用Adam做
num_epoch=30 #迭代次数

for epoch in range(num_epoch):
    epoch_start_time=time.time()
    train_acc=0.0 #计算每个epoch的accuracy和loss
    train_loss=0.0
    val_acc=0.0
    val_loss=0.0
    
model.train() #告诉现在是在模型训练，让参数可以变化，开启dropout等
for i,data in enumerate(train_loader):
    optimizer.zero_grad() #每次使用optimizer前都先将前面存的gradient清零
    #data[0]=x,data[1]=y
    #利用model函数前向传播（forward），算预测值
    train_pred=model(data[0].cuda())
    batch_loss=loss(train_pred,data[1].cuda())
    #用算出来的batch_loss进行反向传播（back propagation）算出每个参数的gradient、
    batch_loss.backward()
    optimizer.step() #用step来更新参数
    #计算最终总的acc和loss
    #np.argmax()返回最大值的索引，axis=1则是对行进行，返回的索引正好就对应了标签，然后和y真实标签比较，则可得到分类正确的数量
    train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
    train_loss += batch_loss.item() #张量中只有一个元素就可以使用item()方法读取

model.eval() #告诉现在是用validation data或testing data评估模型，让算出的参数不变
with torch.no_grad():# 进行验证，当gradient不变时
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())
            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        # 將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
              (epoch + 1, num_epoch, time.time() - epoch_start_time, \
               train_acc / train_set.__len__(), train_loss / train_set.__len__(), val_acc / val_set.__len__(),
               val_loss / val_set.__len__()))

In [ ]:
#由于精确度不高，将traning data和vlidation data合成一个训练集，再次训练
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.001)

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

In [ ]:
#在testing data上跑，并将数据写入csv文件
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)
"""
#保存预测结果
with open("predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))
"""